In [ ]:
from collections import defaultdict
from random import shuffle
from string import ascii_lowercase
from time import time

from wordfreq import get_frequency_dict

from distances import levenshtein
from phonetic_algs import soundex

### Building a dictionary
The frequency dictionary contains words along with their relative frequencies provided by `wordfreq` that are also present in `abc_en` list (concatenated word lists of **A**merican, **B**ritish and **C**anadian English). This combination keeps regional spellings, e.g., *splendor* and *splendour*, *analyze* and *analyse*, and, at the same time, gets rid of words that appear too seldom (have extremely low frequency, like *veery*, + some probable typos: *leik*, *corect* and so on). When dealing solely with non-word errors, we can't afford the presense of such rare words in the dictionary.

In [ ]:
freqd = get_frequency_dict(lang='en', wordlist='large')

\# of words in wordfreq's English list: **302,001**

In [ ]:
dict_path = "abc_en.txt"

with open(dict_path) as f:
    abc = tuple(line.strip().lower() for line in f)

\# of words in abc_en file: **104,224**

**Note!**
The file `abc_en.txt` includes words of [American](https://packages.ubuntu.com/focal/wamerican), [British](https://packages.ubuntu.com/focal/wbritish) and [Canadian English](https://packages.ubuntu.com/focal/wcanadian) taken from the corresponding packages for Ubuntu Linux distibution. You can fetch these three word lists and easily restore the common list as `sorted(set(american + british + canadian))`. 

In [ ]:
freq_d = {k: v for k, v in freqd.items() if k in abc}

**Note!** It makes sense to save `freq_d` (using `json.dump` or other methods).

Total \# of words in the combined dictionary: **75,733**. The keys are lowercased and include different word forms, e.g., plurals and possessives (*cat - cats - cat's*), verbal forms and contractions (*swim - swam - swum - swimming, i'm - she'll - they'd*).

So far, so good. Now, we are going to index these words using the Soundex algorithm.

The results will be stored in another dictionary of the following structure:

`{soundex_index_0: ['tea', 'too', 'two'], soundex_index_1: [], ..., soundex_index_n: [...]}`.

This will save us a bit of time later on.

In [ ]:
sndx_d = defaultdict(list)
length = 8
for word in freq_d:
    sndx_d[soundex(word, length=length)].append(word)

###### Note! Why do we need pronunciation models?

The fact that pronunciation of words tends to affect their (mis)spellings draws attention to letter-to-sound or grapheme-to-morpheme conversions. In linguistic terms, the extent to which a written language deviates from one-to-one letter-phoneme correspondence is described by the concept of *orthographic depth*. In languages with *shallow (transparent) orthographies*, like Finnish, Italian, or Ukrainian, it's easy to pronounce a word based on its spelling. In contrast, *deep (opaque) orthographies* are difficult to read letter-by-letter. French, English, or Swedish words may sound counterintuitively to their spellings due to complex orthographies of these languages. In spelling correction, edit distances, e.g., Levenshtein, work well for languages with small (shallow) orthographic depth. Yet, they don't bode well for languages with deep orthographies, which also require pronunciation being considered.

In [ ]:
# max_ind = len(max(sndx_d.values(), key=len))
# avg_ind = sum(len(v) for v in sndx_d.values()) / len(sndx_d)
# print(len(sndx_d), max_ind, avg_ind)

\# of unique Soundex indexes amounts to **23,143**. The length of 8 symbols is used to reduce the number of words per index. Indexes can be refined further, but as the length grows the biggest number of words in a list levels off (334 per index with default value 4, 191 words -- with length 5+). The average amount of words per index decreases, too. At length 8, it's approximately 3 compared to 16 at the very beginning.
There's a tradeoff between speed and accuracy. Tweak the `length` parameter to see it in action.

### Datasets for training and evaluation
We'll use a [list of common misspellings from Wikipedia](https://en.wikipedia.org/wiki/Wikipedia:Lists_of_common_misspellings/For_machines). See [alternative options](http://www.dcs.bbk.ac.uk/~ROGER/corpora.html). 

First, we transform the original `misspelling->correct_word1[, correct_word_2, ...]` to one-to-one mapping `(misspelling, correct_word)` leaving only the pairs with the correct word in the dictionary. 

In [ ]:
with open('wiki_common_misspellings.txt') as f:
    lines = f.read().splitlines()
            
error_word = [(error, word) for (error, words) in (line.lower().split('->') for line in lines)
                            for word in words.split(', ') if word in freq_d]

In [ ]:
# print(len(error_word))    # 4299 pairs
# print(error_word[17])     # ('abreviation', 'abbreviation')
# # 7 misspellings: ['hten', 'teh', 'tghe', 'ther', 'thge', 'tje', 'tjhe']
# print([error for (error, word) in error_word if word == 'the'])

Now, we can split the set in half for training and evaluation.

In [ ]:
shuffle(error_word)  # the original list was sorted alphabetically
development_set = error_word[:2150]
final_test_set = error_word[2150:]

### Selection mechanism
Given a word `w`, we are trying to find such a correction `c` that, out of all possible candidates, maximizes the probability `P(c|w)`.

In [ ]:
def correct(word, model=spell_1):
    return max(candidates(word, model=model), key=lambda c: freq_d.get(c, 0.0))

In our datasets, all correct words are known. However, if the above function is called on an unknown word, the probability is set to 0.0. Other word frequencies are already stored as probabilities (floating-point numbers between 0 and 1).

### Error model
When generating candidates, we use a massive simplification:

- If a word is in the dictionary, as a correction of itself, it has the highest probability, which is not necessarily so. It sweeps away the second-largest class of spelling errors -- *RWEs*, or *real-word errors*.
- The less is the edit distance to any valid word, the more probable this correction is\* (there are different models below).
- The last and least probable, the word itself, when not listed in the dictionary.

In [ ]:
def candidates(word, model):
    return known([word]) or model(word) or [word]

In [ ]:
def known(words):
    return set(w for w in words if w in freq_d)

The next logical step would be to introduce a noisy channel model, which can tell us how likely each particular misspelling is given the correct word.

*Use weighted edit distance: for any given pair of letters, how likely are different single edits to happen?*

What is really meant by maximizing `P(c|w)` is the following probability product:

`P(c|w) = (P(w|c) * P(c)) / P(w)` (by Bayes' rule)

As `P(w)` is constant, the formula is often shortened to
`P(w|c) * P(c)`,

where `P(w|c)` is a channel model / error model / likelihood,
`P(c)` is a language model / prior.

The noisy channel model can also be counted as the sum of logs of probabilities: `P(c|w) = log(P(w|c)) + b * log(P(c))`, with coefficient `b` to weigh the language model.

Code prototype
```
def noisy_channel(w, b=1.0):
    scores = {}
    for c, e in candidates(w):
        channel = editprob(e)
        prior = freq_d.get(w, 0.0)
        score[c] = log(channel) + b*log(prior)
    return max(scores, key=scores.get)
```

### Candidate generation models

The naive approach of computing the Levenshtein distance between a given word and every dictionary word is quite expensive. Instead, the approach suggested by Peter Norvig in [How to Write a Spelling Corrector](https://norvig.com/spell-correct.html) is used to find similarly spelled words.

In [ ]:
def edits1(word):
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in ascii_lowercase]
    inserts    = [L + c + R               for L, R in splits for c in ascii_lowercase]
    return set(deletes + transposes + replaces + inserts)

In [ ]:
def edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

Now, let's see how we can use this to generate different lists of candidates.

In [ ]:
def spell_1(word):
    """Returns known words with the smallest edit distance of 1, 2."""
    return known(edits1(word)) or known(edits2(word))

def spell_2(word):
    """Returns known words with edit distance of 1 and 2."""
    return known(edits1(word)) | known(edits2(word))

To find words with close pronunciation, we compute the Soundex index for a given word and compare it (yes, using actual Levenshtein distance) with Soundex indices from the dictionary we defined earlier.

Notice that the distance of 0 doesn't correspond to the same word, rather, it matches all words with the same index.

In [ ]:
def sound_0(word):
    """Returns known words with the same Soundex indexes."""
    ind = soundex(word, length=length)
    return sndx_d.get(ind, [])

def sound_1(word):
    """Returns known words with the smallest edit distance of 0, 1 (Soundex indexes)."""
    ind = soundex(word, length=length)
    d0 = sndx_d.get(ind, [])
    d1 = []
    for i in sndx_d:
        d = levenshtein(ind, i)
        if d == 1:
            d1.extend(sndx_d[i])
    return d0 or d1

def sound_2(word):
    """Returns known words with edit distance of 0 and 1 (Soundex indexes)."""
    ind = soundex(word, length=length)
    d01 = sndx_d.get(ind, [])
    for i in sndx_d:
        d = levenshtein(ind, i)
        if d == 1:
            d01.extend(sndx_d[i])
    return d01

Finally, let's combine spelling and pronunciation matching and experiment with this, too.

In [ ]:
def spell_sound_1(word):
    """Returns known words within edit distance 1 or with the same Soundex index."""
    ind = soundex(word, length=length)
    return known(edits1(word)) or sndx_d.get(ind, [])

def spell_sound_2(word):
    """Returns known words within edit distance 1 and with the same Soundex index."""
    ind = soundex(word, length=length)
    return known(edits1(word)) | set(sndx_d.get(ind, []))

### Training

Here are the models we defined:

In [ ]:
models = (spell_1, spell_2, sound_0, sound_1, sound_2, spell_sound_1, spell_sound_2)

In [ ]:
def spelltest(tests, verbose=False):
    "Runs correct(wrong) on all (wrong, right) pairs; reports results."
    for model in models:
        start = time()
        good = 0
        n = len(tests)
        for wrong, right in tests:
            w = correct(wrong, model)
            good += (w == right)
            if w != right and verbose:
                print(f'correction({wrong}) => {w} ({freq_d[w]}); expected {right} ({freq_d[right]})')
        dt = time() - start
        print(f'{model.__name__}: {good/n:.0%} of {n} correct at {n/dt:.0f} words per second')

We can use the development set to select the best model and/or perform error analysis afterwards.

In [ ]:
spelltest(development_set)

The results:

```
spell_1: 83% of 2150 correct at 61 words per second
spell_2: 50% of 2150 correct at 8 words per second
sound_0: 44% of 2150 correct at 88234 words per second
sound_1: 44% of 2150 correct at 2 words per second (Soundex indexes of all test words were in sndx_d)
sound_2: 44% of 2150 correct at 2 words per second
spell_sound_1: 80% of 2150 correct at 6214 words per second
spell_sound_2: 57% of 2150 correct at 5975 words per second
```

The numbers might differ slightly since we shuffled the original dataset.

We can use for test `spell_1`, which returns known words with the smallest edit distance of 1, 2, and `spell_sound_1`, which uses the candidates within edit distance of 1 or words with the same Soundex index. Both models do well; one is better at accuracy, the other is 100 times faster.

### Error analysis & Future work

**Hyphens**

Although `freq_d` contains contractions and possessives (both are written with an apostrophe `'`), the Unix dictionary lacks hyphenated words, e.g., phrasal adjectives like *off-the-shelf*, *all-time* or *eco-friendly*, proper nouns like *Anne-Marie* or *Jean-Paul*, etc. Expanding the dictionary with these and similar words might be of benefit.

**Spaces**

We might also consider inserting spaces in a word (it can be a mistyped phrase!): *aswell* is *as well*, *withe* is *with the*.

For now, the datasets exclude unknown words. Without this restriction, we should deal with the above problems.

**Noisy channel: error model & language model**

As you try different candidate generation models, you can see that the performance of those producing more candidates is significantly worse. This happens because the `correct` function selects the best match based on word frequencies alone (our language model). To make things better, we can try to compute probability for every particular edit (channel model) and introduce a coefficient for prior probability / look at the context, e.g., bigrams, to catch context-sensitive mistakes (language model).

...

### Evaluation

In [ ]:
models = (spell_1, spell_sound_1)
spelltest(final_test_set)

Results:

```
spell_1: 84% of 2149 correct at 55 words per second
spell_sound_1: 80% of 2149 correct at 6217 words per second
```

In [ ]:
def spell_0(word):
    return known(edits1(word))

models = (spell_0,)
spelltest(development_set)
spelltest(final_test_set)

We can check if the candidates in `spell_sound_1` come solely from spellings within edit distance 1 (the Soundex part is not used). As shown below, that's not true (compare to accuracy = 80%).

```
spell_0: 76% of 2150 correct at 5771 words per second
spell_0: 75% of 2149 correct at 6382 words per second
```

In [ ]:
# example = "I'm graat ad everyfing but speling".lower().split()

# print(*[correct(word, model=spell_1) for word in example])
# print(*[correct(word, model=spell_sound_1) for word in example])